# NOTEBOOK POUR LA SELECION DU MODELE, DE CONFIGURATION DE MLFLOW 


## IMPORTS ET CONFIGURATION DE MLFLOW, DEFINITION DES EPERIENCES...


In [27]:
import mlflow
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Configuration de MLflow
current_dir = os.getcwd()
mlflow_dir = os.path.join(current_dir, 'mlruns')
mlflow.set_tracking_uri(f"file:{mlflow_dir}")

# Créer le dossier mlruns et le sous-dossier .trash
os.makedirs(os.path.join(mlflow_dir, '.trash'), exist_ok=True)

# Définir l'expérience
experiment = mlflow.set_experiment("Classification de texte")

print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Experiment Name: {experiment.name}")

2024/09/28 21:11:09 INFO mlflow.tracking.fluent: Experiment with name 'Classification de texte' does not exist. Creating a new experiment.


MLflow Version: 2.16.2
Tracking URI: file:/home/utilisateur/Documents/dev/devia/API_ML/models/mlruns
Experiment ID: 650145881295194294
Experiment Name: Classification de texte


### Chargement des données et division en ensembles d'entraînement et de test

In [ ]:
df = pd.read_csv('data.csv', encoding='latin-1', sep=';')
print(df.head())


X = df['Description']
y = df['TAG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Definition des pipelines et des grilles de parametres

In [29]:
pipelines = {
    'SVM': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', SVC())
    ]),
    'RandomForest': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', RandomForestClassifier())
    ]),
    'GradientBoosting': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', GradientBoostingClassifier())
    ]),
    'NaiveBayes': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
    ]),
    'LogisticRegression': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ])
}

param_grids = {
    'SVM': {
        'tfidf__max_features': [5000, 10000],
        'clf__C': [0.1, 1, 10],
        'clf__kernel': ['linear', 'rbf']
    },
    'RandomForest': {
        'tfidf__max_features': [5000, 10000],
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [10, 20, None]
    },
    'GradientBoosting': {
        'tfidf__max_features': [5000, 10000],
        'clf__n_estimators': [100, 200],
        'clf__learning_rate': [0.01, 0.1]
    },
    'NaiveBayes': {
        'tfidf__max_features': [5000, 10000],
        'clf__alpha': [0.1, 1.0]
    },
    'LogisticRegression': {
        'tfidf__max_features': [5000, 10000],
        'clf__C': [0.1, 1, 10],
        'clf__penalty': ['l1', 'l2']
    }
}

### ENTRAINEMENT DES MODELES ET ENREGISTREMENT DANS MLFLOW

In [32]:
for name, pipeline in pipelines.items():
    with mlflow.start_run(run_name=name):
        print(f"Entraînement du modèle : {name}")
        
        # Recherche d'hyperparamètres
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)
        
        # Meilleur modèle
        best_model = grid_search.best_estimator_
        
        # Prédictions sur l'ensemble de test
        y_test_pred = best_model.predict(X_test)
        
        # Prédictions sur l'ensemble d'entraînement
        y_train_pred = best_model.predict(X_train)
        
        # Calcul des métriques pour l'ensemble de test
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred, average='weighted')
        test_recall = recall_score(y_test, y_test_pred, average='weighted')
        test_f1 = f1_score(y_test, y_test_pred, average='weighted')
        
        # Calcul des métriques pour l'ensemble d'entraînement
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred, average='weighted')
        train_recall = recall_score(y_train, y_train_pred, average='weighted')
        train_f1 = f1_score(y_train, y_train_pred, average='weighted')
        
        # Enregistrement dans MLflow
        mlflow.log_param("model", name)
        mlflow.log_params(grid_search.best_params_)
        
        # Métriques de test
        mlflow.log_metric("test_accuracy", test_accuracy)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_f1", test_f1)
        
        # Métriques d'entraînement
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("train_f1", train_f1)
        
        # Enregistrement du meilleur modèle
        mlflow.sklearn.log_model(best_model, "model")
        
        # Enregistrement de la matrice de confusion
        cm = confusion_matrix(y_test, y_test_pred)
        cm_df = pd.DataFrame(cm, index=best_model.classes_, columns=best_model.classes_)
        mlflow.log_dict(cm_df.to_dict(), "confusion_matrix.json")
        
        print(f"Meilleurs paramètres pour {name}: {grid_search.best_params_}")
        print(f"Précision de test: {test_accuracy:.4f}")
        print(f"Précision d'entraînement: {train_accuracy:.4f}")
        print("-" * 50)

print("Entraînement terminé. Vérifiez MLflow UI pour les résultats détaillés.")

Entraînement du modèle : SVM
Fitting 5 folds for each of 12 candidates, totalling 60 fits


2024/09/28 21:12:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres pour SVM: {'clf__C': 10, 'clf__kernel': 'linear', 'tfidf__max_features': 5000}
Précision de test: 0.7500
Précision d'entraînement: 1.0000
--------------------------------------------------
Entraînement du modèle : RandomForest
Fitting 5 folds for each of 12 candidates, totalling 60 fits


2024/09/28 21:12:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres pour RandomForest: {'clf__max_depth': 10, 'clf__n_estimators': 100, 'tfidf__max_features': 10000}
Précision de test: 0.8500
Précision d'entraînement: 1.0000
--------------------------------------------------
Entraînement du modèle : GradientBoosting
Fitting 5 folds for each of 8 candidates, totalling 40 fits


...

Meilleurs paramètres pour GradientBoosting: {'clf__learning_rate': 0.01, 'clf__n_estimators': 100, 'tfidf__max_features': 10000}
Précision de test: 0.8000
Précision d'entraînement: 1.0000
--------------------------------------------------
Entraînement du modèle : NaiveBayes
Fitting 5 folds for each of 4 candidates, totalling 20 fits


2024/09/28 21:12:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres pour NaiveBayes: {'clf__alpha': 1.0, 'tfidf__max_features': 5000}
Précision de test: 0.7500
Précision d'entraînement: 1.0000
--------------------------------------------------
Entraînement du modèle : LogisticRegression
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/utilisateur/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/home/utilisateur/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/utilisateur/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/utilisateur/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/pipeline.py", line 473

Meilleurs paramètres pour LogisticRegression: {'clf__C': 10, 'clf__penalty': 'l2', 'tfidf__max_features': 5000}
Précision de test: 0.8000
Précision d'entraînement: 1.0000
--------------------------------------------------
Entraînement terminé. Vérifiez MLflow UI pour les résultats détaillés.
